# 100k events ; using CKKW

In [4]:
import ROOT
import math
import numpy as np

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)

def calculate_delta_phi_jet_met(jet_phi, met_phi): 
    return delta_phi(jet_phi, met_phi)

def analyze_specific_points(root_file_path, cross_section):
    # Create chain of root trees
    chain = ROOT.TChain("Delphes")
    chain.Add(root_file_path)

    # Create object of class ExRootTreeReader
    treeReader = ROOT.ExRootTreeReader(chain)
    numberOfEntries = treeReader.GetEntries()

    # Get pointer to the branch containing Particle information
    branchJet = treeReader.UseBranch("Jet")
    branchMET = treeReader.UseBranch("MissingET")
    branchEvent = treeReader.UseBranch("Weight")

    # Events Counter
    initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0
    initial_events_sq = events_passed_met_cut_sq = events_passed_pt_eta_cut_sq = events_passed_jets_criteria_sq = events_passed_delta_phi_cut_sq = events_passed_leading_jet_cut_sq = 0

    #### Initial Cuts 
    pt_cut = 30.0
    max_eta = 2.8  

    # Constants for scaling
    luminosity = 140.0 * 1000  # pb # https://twiki.cern.ch/twiki/bin/view/LHCPhysics/SUSYCrossSections13TeVgluglu
    # cross_section = 0.385  # pb for gluino at 1 TeV

    # Dictionary to store MET cut values and corresponding event counts
    met_cuts = {200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

    # Initialize dictionary to store squared sum of event weights for each MET cut value
    met_cuts_sq = {met_cut: 0 for met_cut in met_cuts}

    sum_event_weights = 0
    for i in range(numberOfEntries):
        treeReader.ReadEntry(i)
        sum_event_weights += branchEvent.At(6).Weight

    # Loop over all events
    for entry in range(numberOfEntries):
        # Load selected branches with data from specified event
        treeReader.ReadEntry(entry)

        num_jets_meeting_criteria = leading_jet_pt = leading_jet_eta = 0

        initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
        initial_events_sq += (branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights) ** 2

        if branchMET.At(0).MET > 200:
            events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            events_passed_met_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

            # Loop over all jets
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)

                # Jet Pt and Eta Cuts
                if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                    num_jets_meeting_criteria += 1

                # Save Leading Jet Pt and Eta
                if jet.PT > leading_jet_pt:
                    leading_jet_pt = jet.PT
                    leading_jet_eta = jet.Eta

            # Count events passing pt, eta cut
            if num_jets_meeting_criteria > 0:
                events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                events_passed_pt_eta_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

            # To set limit over jets
            if num_jets_meeting_criteria <= 4:
                events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                events_passed_jets_criteria_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

                # Delta_Phi cut implementation
                delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
                delta_phis = []
                for jet_index in range(branchJet.GetEntries()):
                    jet = branchJet.At(jet_index)
                    delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                    delta_phis.append(delta_phi_jet_met)

                # Check how many jets pass the delta_phi cut
                if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                    events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                    events_passed_delta_phi_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

                    # Leading Jets Pt and Eta cuts
                    if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                        events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                        events_passed_leading_jet_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

                        # Loop over MET cut values
                        for met_cut in met_cuts:
                            if branchMET.At(0).MET > met_cut:
                                met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                                met_cuts_sq[met_cut] += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2
                                

    print(f"\nAnalysis results for root file: {root_file_path}")
    print("===================================================")

    # Calculate statistical uncertainties for each cut
    initial_events_uncer = np.sqrt(initial_events_sq)
    events_passed_met_cut_uncer = np.sqrt(events_passed_met_cut_sq)
    events_passed_pt_eta_cut_uncer = np.sqrt(events_passed_pt_eta_cut_sq)
    events_passed_jets_criteria_uncer = np.sqrt(events_passed_jets_criteria_sq)
    events_passed_delta_phi_cut_uncer = np.sqrt(events_passed_delta_phi_cut_sq)
    events_passed_leading_jet_cut_uncer = np.sqrt(events_passed_leading_jet_cut_sq)

    print(f"\nInitial number of events: {initial_events:.2f} +/- {initial_events_uncer:.2f} ")
    print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f} +/- {events_passed_met_cut_uncer:.2f}")
    print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f} +/- {events_passed_pt_eta_cut_uncer:.2f}")
    print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f} +/- {events_passed_jets_criteria_uncer:.2f}")
    print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f} +/- {events_passed_delta_phi_cut_uncer:.2f}")
    print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f} +/- {events_passed_leading_jet_cut_uncer:.2f}")

    paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                      600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

    ratios = {}  # Dictionary to store the ratios
    ratios_uncer = {} # Dictionary to store the ratios uncertainty
    
    # Calculate the ratio for each MET cut value
    for met_cut in met_cuts:
        if met_cut in paper_met_cuts:
            ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
            ratio_uncer = np.sqrt(met_cuts_sq[met_cut]) / paper_met_cuts[met_cut]
            ratios[met_cut] = ratio
            ratios_uncer[met_cut] = ratio_uncer
            print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f} +/- {np.sqrt(met_cuts_sq[met_cut]):.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f} +/- {ratio_uncer:.2f}")
            

    # Find the MET cut with the maximum ratio
    max_ratio_met_cut = max(ratios, key=ratios.get)

    # Calculate gluino mass point
    gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])

    print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", format(met_cuts[max_ratio_met_cut],'5f'), ") =", format(gluino_mass_point, '5f'))

    uncertainty_max_ratio_met_cut = np.sqrt(met_cuts_sq[max_ratio_met_cut])

    # Calculate gluino mass point with uncertainty when adding the uncertainty to MET cut
    gluino_mass_point_plus_uncertainty = cross_section * (paper_met_cuts[max_ratio_met_cut] / (met_cuts[max_ratio_met_cut] - uncertainty_max_ratio_met_cut))
    print("\nWhen adding the uncertainty to MET cut, the maximum gluino mass point can be around:", format(gluino_mass_point_plus_uncertainty, '.5f'))

    # Calculate gluino mass point with uncertainty when subtracting the uncertainty from MET cut
    gluino_mass_point_minus_uncertainty = cross_section * (paper_met_cuts[max_ratio_met_cut] / (met_cuts[max_ratio_met_cut] + uncertainty_max_ratio_met_cut))
    print("When subtracting the uncertainty from MET cut, the maximum gluino mass point can be around:", format(gluino_mass_point_minus_uncertainty, '.5f'))


# no pile-up, just the ordinary ATLAS card

In [5]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-990-1000/Events/run_100k/tag_1_delphes_events.root", 0.385)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-990-1000/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 195.06 

Number of events passed MET > 200 GeV cut: 13549.63 +/- 102.51

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13524.41 +/- 102.42

Number of events passed number of jets <= 4 cut: 11443.87 +/- 93.98

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9223.06 +/- 84.21

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7988.20 +/- 78.83

For SR MET > 200 GeV: S_Exp: 7988.20 +/- 78.83, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6318.24 +/- 70.78, S_Obs: 41158, and ratio: 0.15 +/- 0.00

For SR MET > 300 GeV: S_Exp: 4827.87 +/- 62.14, S_Obs: 20893, and ratio: 0.23 +/- 0.00

For SR MET > 350 GeV: S_Exp: 3643.84 +/- 53.98, S_Obs: 11937, and ratio: 0.31 +/- 0.00

For SR MET > 400 GeV: S_Exp: 2776.32 +/- 47.0

# pile-up ATLAS card, but default values for energy resolution

In [6]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-990-1000_newPileUp/Events/run_100k/tag_1_delphes_events.root", 0.385)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-990-1000_newPileUp/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 195.06 

Number of events passed MET > 200 GeV cut: 15015.02 +/- 107.28

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 14989.51 +/- 107.20

Number of events passed number of jets <= 4 cut: 12277.87 +/- 96.72

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 7273.71 +/- 74.36

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 5858.45 +/- 67.37

For SR MET > 200 GeV: S_Exp: 5858.45 +/- 67.37, S_Obs: 102274, and ratio: 0.06 +/- 0.00

For SR MET > 250 GeV: S_Exp: 4867.22 +/- 61.87, S_Obs: 41158, and ratio: 0.12 +/- 0.00

For SR MET > 300 GeV: S_Exp: 3762.93 +/- 54.73, S_Obs: 20893, and ratio: 0.18 +/- 0.00

For SR MET > 350 GeV: S_Exp: 2832.73 +/- 47.58, S_Obs: 11937, and ratio: 0.24 +/- 0.00

For SR MET > 400 GeV: S_Exp: 2169.7

# pile-up ATLAS card, but modified values for energy resolution

In [7]:
analyze_specific_points("/data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-990-1000_newPileUp-ATLAS-default/Events/run_100k/tag_1_delphes_events.root", 0.385)


Analysis results for root file: /data/analysis/iduminic/MSSM-gluino-neutralino/MSSM_LO_gogo_TESTCKKW_V2-990-1000_newPileUp-ATLAS-default/Events/run_100k/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 195.06 

Number of events passed MET > 200 GeV cut: 15032.83 +/- 107.34

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 15020.46 +/- 107.29

Number of events passed number of jets <= 4 cut: 12283.40 +/- 96.76

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 7222.73 +/- 74.14

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 5801.74 +/- 67.09

For SR MET > 200 GeV: S_Exp: 5801.74 +/- 67.09, S_Obs: 102274, and ratio: 0.06 +/- 0.00

For SR MET > 250 GeV: S_Exp: 4845.02 +/- 61.77, S_Obs: 41158, and ratio: 0.12 +/- 0.00

For SR MET > 300 GeV: S_Exp: 3725.27 +/- 54.49, S_Obs: 20893, and ratio: 0.18 +/- 0.00

For SR MET > 350 GeV: S_Exp: 2829.46 +/- 47.57, S_Obs: 11937, and ratio: 0.24 +/- 0.00

For SR MET > 400 GeV: